# Day 3 - Get the Weak Lensing Signals

### Reading data from the catalog and appying the selection cuts.

In [1]:
def lens_select(zmin=0.1, zmax=0.33, lammin=55, lammax=100):
    #please check the file path properly 
    data = pd.read_csv('./DataStore/redmapper.dat', delim_whitespace=1)
    #sample selection cut
    idx  = (data['lambda']>lammin) & (data['lambda']<=lammax)
    idx  = idx & (data['zred']>zmin) & (data['zred']<=zmax)
    ra   = data['ra'].values[idx]
    dec  = data['dec'].values[idx]
    zred = data['zred'].values[idx]
    #as we have no weights to apply we set them to unity
    wgt  = ra*1.0/ra
    print('number of lenses=%d'%len(ra))
    return ra, dec, zred, wgt


In [2]:
def read_sources(ifil):
    # various columns in sources 
    # ragal, decgal, e1gal, e2gal, wgal, rms_egal, mgal, c1gal, c2gal, R2gal, zphotgal
    data = pd.read_csv(ifil, delim_whitespace=1).values
    zphotgal = data[:,-1]
    # sanity checks on the sources data
    idx = (np.sum(np.isnan(data), axis=1)==0) &  (zphotgal>0)
    datagal = np.zeros((np.sum(idx),7))
    datagal[:,:6] = data[idx,:6]
    datagal[:,6]  = data[idx,-1]
    # collects only -  ragal, decgal, e1gal, e2gal, wgal, rms_egal, zphotgal
    return datagal

### Computing the tangential shear $e_{\rm t}$ and inverse critical density $\Sigma^{-1}_{\rm crit}$ 

In [3]:
def get_et(lra, ldec, sra, sdec, se1, se2):
    lra  = lra*np.pi/180
    ldec = ldec*np.pi/180
    sra  = sra*np.pi/180
    sdec = sdec*np.pi/180

    c_theta = np.cos(ldec)*np.cos(sdec)*np.cos(lra - sra) + np.sin(ldec)*np.sin(sdec)
    s_theta = np.sqrt(1-c_theta**2)

    # phi to get the compute the tangential shear
    c_phi   = np.cos(ldec)*np.sin(sra - lra)*1.0/s_theta
    s_phi   = (-np.sin(ldec)*np.cos(sdec) + np.cos(ldec)*np.cos(sra - lra)*np.sin(sdec))*1.0/s_theta
    # tangential shear
    e_t     = - se1*(2*c_phi**2 -1) - se2*(2*c_phi * s_phi)

    return e_t


In [4]:
def get_sigma_crit_inv(lzred, szred, cc):
    # some important constants for the sigma crit computations
    gee = 4.301e-9 #km^2 Mpc M_sun^-1 s^-2 gravitational constant
    cee = 3e5 #km s^-1
    # sigma_crit_calculations for a given lense-source pair
    sigm_crit_inv = cc.angular_diameter_distance(lzred).value * cc.angular_diameter_distance_z1z2(lzred, szred).value * (1.0 + lzred)**2 * 1.0/cc.angular_diameter_distance(szred).value
    sigm_crit_inv = sigm_crit_inv * 4*np.pi*gee*1.0/cee**2 
    sigm_crit_inv = 1e12*sigm_crit_inv #esd's are in pc not in Mpc

    return sigm_crit_inv


### $\Delta \Sigma (R)$ measurements and writing the output to a file

In [6]:
def run_pipe(Omegam=0.315, rmin=0.2, rmax=2.0, nbins=10, zdiff=0.4):
    #set the cosmology with omegaM parameter 
    cc = FlatLambdaCDM(H0=100, Om0=Omegam) # fixing H0=100 to set units in Mpc h-1
    
    # set the projected radial binning 
    rmin  =  rmin
    rmax  =  rmax
    nbins = nbins #10 radial bins for our case
    rbins  = np.logspace(np.log10(rmin), np.log10(rmax), nbins + 1)
    rdiff  = np.log10(rbins[1]*1.0/rbins[0])
 
    # initializing arrays for signal compuations
    sumdsig_num   = np.zeros(len(rbins[:-1]))
    sumdsigsq_num = np.zeros(len(rbins[:-1]))
    sumwls        = np.zeros(len(rbins[:-1]))
    sumwls_resp   = np.zeros(len(rbins[:-1]))

    # getting the lenses data
    lra, ldec, lred, lwgt = lens_select()

    # convert lense ra and dec into x,y,z cartesian coordinates
    lx, ly, lz = get_xyz(lra, ldec)
     
    # putting kd tree around the lenses
    lens_tree = KDTree(np.array([lx, ly, lz]).T)
    
    
    print('lenses tree is ready\n')
    
    # setting maximum search radius
    dcommin = cc.comoving_distance(np.min(lred)).value
    dismax  = (rmax*1.0/(dcommin)) 

    # lets first catch the file list for sources
    sflist = np.sort(glob.glob('./DataStore/hsc/*.dat'))

    # Ready to pounce on the source data
    for ifil in sflist:
        # catching the source data matrix
        # please have a check for the columns names
        datagal = read_sources(ifil)
        Ngal = len(datagal[:,0])  # total number of galaxies in the source file
        # first two entries are ra and dec for the sources
        allragal  = datagal[:,0]
        alldecgal = datagal[:,1]
        # ra and dec to x,y,z for sources
        allsx, allsy, allsz = get_xyz(allragal, alldecgal)
        # query in a ball around individual sources and collect the lenses ids with a maximum radius
        slidx = lens_tree.query_ball_point(np.transpose([allsx, allsy, allsz]), dismax) 
        # various columns in sources 
        # ragal, decgal, e1gal, e2gal, wgal, rms_egal, mgal, c1gal, c2gal, R2gal, zphotgal
        # looping over all the galaxies
        for igal in range(Ngal):    
            ragal    = datagal[igal,0]
            decgal   = datagal[igal,1]
            e1gal    = datagal[igal,2]
            e2gal    = datagal[igal,3]
            wgal     = datagal[igal,4]
            rms_egal = datagal[igal,5]
            zphotgal = datagal[igal,6]
           
            # array of lenses indices
            lidx = np.array(slidx[igal])
            # removing sources which doesn't have any lenses around them 
            if len(lidx)==0:
                continue
           
            # selecting a cleaner background
            zcut = (lred[lidx] < (zphotgal - zdiff)) #only taking the foreground lenses
            # again skipping the onces which doesn't satisfy the above criteria
            if np.sum(zcut)==0.0:
                continue
            # collecting the  data of lenses around individual source
            lidx   = lidx[zcut] # this will catch the array indices for our lenses
            sra    = ragal
            sdec   = decgal
            
            l_ra   = lra[lidx]
            l_dec  = ldec[lidx]
            l_zred = lred[lidx] 
            l_wgt  = lwgt[lidx] 
           
            sx, sy, sz = get_xyz(sra,sdec) # individual galaxy ra,dec-->x,y,z
            lx, ly, lz = get_xyz(l_ra,l_dec) # individual galaxy ra,dec-->x,y,z
            
            # getting the radial separations for a lense source pair 
            sl_sep = np.sqrt((lx - sx)**2 + (ly - sy)**2 + (lz - sz)**2)
            sl_sep = sl_sep * cc.comoving_distance(l_zred).value
            for ll,sep in enumerate(sl_sep):
                if sep<rmin or sep>rmax:
                    continue
                rb = int(np.log10(sep*1.0/rmin)*1/rdiff)
               
                # get tangantial components given positions and shapes
                e_t = get_et(lra = l_ra[ll], ldec = l_dec[ll], sra = sra, sdec = sdec, se1 = e1gal,  se2 = e2gal)

                # sigma_crit_calculations for a given lense-source pair
                sigm_crit_inv = get_sigma_crit_inv(l_zred[ll], zphotgal, cc)

                # following equations given in the surhud's lectures 
                w_ls    = l_wgt[ll] * wgal * (sigm_crit_inv)**2
                w_ls_by_av_sigc_inv = l_wgt[ll] * wgal * sigm_crit_inv

                # separate numerator and denominator computation   
                sumdsig_num[rb]   += w_ls_by_av_sigc_inv  * e_t
                sumdsigsq_num[rb] += (w_ls_by_av_sigc_inv * e_t)**2
                sumwls[rb]        += w_ls
                sumwls_resp[rb]   += w_ls * (1-rms_egal**2)

        print(ifil)
        
    outputfile = 'iagrg_dsigma.dat_no_res'  
    fout = open(outputfile, "w")
    fout.write("# 0:rmin/2+rmax/2 1:DeltaSigma  2:SN_ErrDeltaSigma\n")
    for i in range(len(rbins[:-1])):
        rrmin = rbins[i]
        rrmax = rbins[i+1]
        Resp = sumwls_resp[i]*1.0/sumwls[i]
        
        fout.write("%le\t%le\t%le\n"%(rrmin/2.0+rrmax/2.0, sumdsig_num[i]*1.0/sumwls[i]/2./Resp, np.sqrt(sumdsigsq_num[i])*1.0/sumwls[i]/2./Resp))
    fout.write("#OK")    
    fout.close()
    
    return 0

### Plotting the  weak lensing signal